# SETTING ENVIRONMENT


In [35]:
"""# mount the colab with google drive
from google.colab import drive
drive.mount('/content/drive')"""

"# mount the colab with google drive\nfrom google.colab import drive\ndrive.mount('/content/drive')"

In [36]:
# set folder tempat kerja (current working directory)
import os
cwd = "/Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita"
#cwd = '/content/drive/MyDrive/Monitoring Berita'
os.chdir(cwd)

# PROSES

## Baca Data


In [37]:
import json
from pathlib import Path

cfg_path = f"{cwd}/config.json"
CONFIG_PATH = Path(cfg_path)

with open(CONFIG_PATH, "r", encoding="utf-8") as f:
            cfg = json.load(f)

INPUT_EXCEL_PATH = cfg["labelled_data_xlsx"]

In [38]:
import pandas as pd
df_input = pd.read_excel(INPUT_EXCEL_PATH)

In [39]:
df = df_input[['judul_berita', 'source_domain', 'url_berita', 'tanggal_berita_norm', 'kategori_isu',
               'artikel_berita_bersih']]

In [40]:
df = df[df['kategori_isu'] != "Isu Lainnya"].reset_index(drop=True)

In [41]:
# test untuk 10 data
df = df.sample(10, random_state=42).reset_index(drop=True)

# MAIN

In [42]:
# ==============================
# 1. Import & setup
# ==============================
import os
import re
import json
import time
import logging
import pandas as pd
from typing import Dict, Any, List

# Setup logging agar progress terlihat
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
)

from tqdm.auto import tqdm
tqdm.pandas()  # memungkinkan .progress_apply di pandas jika diperlukan


In [43]:
# Load API keys from mac environment variables (and ensure they are available to HTTP clients)
import os
import json
from pathlib import Path

DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Optional: try to read from config.json if env vars are missing
if not DEEPSEEK_API_KEY or not OPENAI_API_KEY:
    cfg_file = Path("config.json")
    if cfg_file.exists():
        try:
            with open(cfg_file, "r", encoding="utf-8") as f:
                cfg_local = json.load(f)
            DEEPSEEK_API_KEY = DEEPSEEK_API_KEY or cfg_local.get("deepseek_api_key") or cfg_local.get("deepseek_api")
            OPENAI_API_KEY = OPENAI_API_KEY or cfg_local.get("openai_api_key") or cfg_local.get("openai_api")
        except Exception:
            pass

# Ensure os.environ is populated for downstream HTTP calls
if DEEPSEEK_API_KEY:
    os.environ["DEEPSEEK_API_KEY"] = DEEPSEEK_API_KEY
if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

print(
    "API keys - Deepseek:", "OK" if os.getenv("DEEPSEEK_API_KEY") else "MISSING",
    "| OpenAI:", "OK" if os.getenv("OPENAI_API_KEY") else "MISSING"
)

API keys - Deepseek: OK | OpenAI: OK


In [44]:
# ==============================
# 2. Fungsi cleaning teks
# ==============================

def normalize_whitespace(text: str) -> str:
    """Normalisasi whitespace: hapus spasi ganda, spasi di pinggir, newline berlebih."""
    if not isinstance(text, str):
        return ""
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

def strip_non_text_noise(text: str) -> str:
    """Hilangkan pola umum yang bukan isi artikel (disclaimer, navigasi, dsb)."""
    if not text:
        return ""
    text = re.sub(r"^\s*[-•\u2022]\s*", "", text, flags=re.M)
    noise_patterns = [
        r"Bagikan:\s*Facebook.*?$",
        r"Follow\s+kami.*?$",
        r"Editor:\s.*?$",
        r"Penulis:\s.*?$",
    ]
    for pat in noise_patterns:
        text = re.sub(pat, "", text, flags=re.I | re.M)
    return text

def final_cleanup(text: str, min_line_len: int = 3) -> str:
    """Pembersihan akhir: normalisasi whitespace, buang baris terlalu pendek."""
    text = normalize_whitespace(text)
    lines = [ln for ln in text.splitlines() if len(ln.strip()) >= min_line_len]
    return "\n".join(lines).strip()

def clean_article(text: str) -> str:
    """Pipeline cleaning lengkap."""
    if not isinstance(text, str):
        return ""
    text = strip_non_text_noise(text)
    text = final_cleanup(text)
    return text


In [45]:
# ==============================
# 3. Prompt & parsing JSON
# ==============================

# System prompt untuk membatasi gaya output LLM agar selalu JSON valid
SYSTEM_PROMPT = (
"Anda adalah asisten analisis berita.  "
"Keluarkan **hanya** JSON valid sesuai skema yang ditentukan, **tanpa penjelasan tambahan**."  
"Jika ada bagian yang tidak dapat ditentukan, isilah dengan string kosong `""`."
)

USER_TEMPLATE = """TUGAS:
TUGAS:
1) Tentukan nilai **"topik"** (pilih satu dari kategori berikut):
   - Kemenkeu  
   - Nasional  
   - Internasional  
   - Lainnya  

   “Kemenkeu” digunakan jika berita berkaitan langsung dengan tugas/fungsi Kementerian Keuangan RI.  
   “Nasional” untuk kebijakan pusat selain Kemenkeu.  
   “Internasional” untuk hubungan luar negeri atau isu global.  
   “Lainnya” untuk kategori yang tidak sesuai ketiganya (misalnya olahraga, hiburan, dsb.).

2) Tentukan **"subtopik"**: frasa singkat yang menggambarkan isi pokok berita (misalnya “reformasi pajak”, “utang luar negeri”, “kerjasama bilateral”, “skandal korupsi”, dsb.).

3) Tentukan **"sentimen"** (pilih salah satu):  
   - positif  
   - netral  
   - negatif  

4) Beri **"alasan_sentimen"**: penjelasan singkat (maksimal 2 kalimat) kenapa sentimen tersebut sesuai.

5) Temukan **"poin_of_interest"**: nama pejabat / public figure (dengan jabatan dan gelar jika ada) yang menjadi fokus dalam berita (jika ada).

6) Ambil **"statement_pejabat"**: **semua kutipan** langsung dari berita, persis seperti tertulis (jangan diubah atau ringkas).

7) Beri **"confidence"**: angka antara 0.0 hingga 1.0 yang menunjukkan tingkat kepastianmu terhadap hasil tersebut.

ATURAN-SENTIMEN TAMBAHAN (jika relevan):
- Kata “utang” atau “defisit” dalam konteks pembiayaan → **positif**  
- “Anjing pelacak” → **positif** (jika digunakan dalam konteks penegakan hukum)  
- “Penyelundupan” + konteks Bea Cukai / DJBC → **positif**  
- “Penggelapan pajak” + DJP / Direktorat Jenderal Pajak → **positif**  
- “Pajak” atau “cukai” tanpa konteks penindakan → **netral**  
- “Korupsi” + KPK / Komisi Pemberantasan Korupsi → **positif**  
- “Strategi penyerapan anggaran yang efektif” → **positif**  

FORMAT OUTPUT HARUS JSON (tanpa tambahan teks apa pun), contoh:


Output HARUS JSON:
{{
  "topik": "Kemenkeu|Nasional|Internasional|Lainnya",
  "subtopik": "string",
  "sentimen": "positif|netral|negatif",
  "alasan_sentimen": "string",
  "poin_of_interest": "string",
  "statement_pejabat": "string",
  "confidence": 0.0
}}

TEKS_BERITA:
"{text}"
"""

def parse_first_json_block(s: str) -> Dict[str, Any]:
    """Ambil JSON {...} pertama dari string dan parse jadi dict."""
    if not isinstance(s, str):
        raise ValueError("Response bukan string.")
    m = re.search(r"\{.*\}", s, flags=re.S)
    if not m:
        raise ValueError("Tidak ditemukan blok JSON dalam respons.")
    snippet = m.group(0)
    try:
        return json.loads(snippet)
    except json.JSONDecodeError:
        fixed = snippet.replace("“", '"').replace("”", '"').replace("’", "'")
        return json.loads(fixed)

In [46]:
# ==============================
# 4. Provider: Mock, OpenAI, DeepSeek
# ==============================

# Optimasi DeepSeek:
# - Reuse HTTP session (keep-alive)
# - Turunkan max_tokens karena output JSON kecil
# - Siapkan fungsi helper agar mudah ganti parameter

_DEEPSEEK_SESSION = None

def _get_deepseek_session():
    global _DEEPSEEK_SESSION
    if _DEEPSEEK_SESSION is None:
        import requests
        _DEEPSEEK_SESSION = requests.Session()
    return _DEEPSEEK_SESSION


def call_mock_classifier(text: str) -> Dict[str, Any]:
    """Classifier dummy untuk demo/offline."""
    t = (text or "").lower()
    if any(k in t for k in ["apbn", "pajak", "kemenkeu", "sri mulyani", "bea cukai"]):
        topik = "Kemenkeu"
    elif any(k in t for k in ["presiden", "dpr", "kpk", "polri", "politik"]):
        topik = "Nasional"
    else:
        topik = "Lainnya"

    if any(k in t for k in ["skandal", "korupsi", "anjlok", "utang"]):
        sentimen = "negatif"; alasan = "Berita bernuansa kritik atau masalah."
    elif any(k in t for k in ["subsidi", "bantuan", "pemulihan", "peningkatan"]):
        sentimen = "positif"; alasan = "Berita menekankan capaian atau dukungan."
    else:
        sentimen = "netral"; alasan = "Berita informatif tanpa penilaian."

    return {
        "topik": topik,
        "subtopik": "",
        "sentimen": sentimen,
        "alasan_sentimen": alasan,
        "confidence": 0.6,
    }


def call_openai(messages: List[Dict[str, str]], model: str = "gpt-5-mini") -> str:
    try:
        from openai import OpenAI
    except ImportError as e:
        raise RuntimeError("Library openai belum terpasang. Install dengan: pip install openai") from e
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        raise RuntimeError("OPENAI_API_KEY tidak ditemukan di environment.")
    client = OpenAI(api_key=api_key)
    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.0,
        max_completion_tokens=400,  # cukup untuk JSON kecil
        top_p=1,
    )
    return resp.choices[0].message.content or ""


def call_deepseek(messages: List[Dict[str, str]], model: str = "deepseek-chat") -> str:
    import requests
    import math
    api_key = os.getenv('DEEPSEEK_API_KEY','')
    if not api_key:
        raise RuntimeError("DEEPSEEK_API_KEY tidak ditemukan di environment.")

    # Heuristik: output JSON pendek, jadi max_tokens tidak perlu besar (hemat & cepat)
    # Jika prompt panjang (>5k chars) naikkan sedikit.
    user_content = "".join(m.get("content","") for m in messages if m.get("role")=='user')
    base_max = 600
    if len(user_content) > 6000:
        base_max = 800
    elif len(user_content) > 3000:
        base_max = 450

    url = "https://api.deepseek.com/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": model,
        "messages": messages,
        "temperature": 0.0,
        "max_tokens": base_max,
        "top_p": 1,
    }
    session = _get_deepseek_session()
    # Timeout lebih agresif agar cepat gagal & bisa retry di wrapper
    r = session.post(url, headers=headers, data=json.dumps(payload), timeout=45)
    r.raise_for_status()
    data = r.json()
    try:
        return data["choices"][0]["message"]["content"]
    except Exception:
        raise RuntimeError(f"Format respons DeepSeek tidak terduga: {data}")

In [47]:
# ==============================
# 5. Klasifikasi wrapper (retry & fallback) + Model Registry
# ==============================

# Registry untuk memudahkan penambahan provider baru
MODEL_REGISTRY = {
    "mock": {
        "default_model": "mock-local",
        "caller": lambda messages, model: json.dumps(call_mock_classifier(messages[-1]['content']) if messages else {})
    },
    "openai": {
        "default_model": "gpt-4o-mini",  # mudah diganti nanti
        "caller": lambda messages, model: call_openai(messages, model=model)
    },
    "deepseek": {
        "default_model": "deepseek-chat",
        "caller": lambda messages, model: call_deepseek(messages, model=model)
    },
}

def get_provider_config(provider: str):
    prov = (provider or "mock").lower()
    if prov not in MODEL_REGISTRY:
        logging.warning(f"Provider '{provider}' tidak dikenal. Fallback ke 'mock'.")
        prov = "mock"
    return prov, MODEL_REGISTRY[prov]


def classify_with_llm(
    text: str,
    provider: str = "mock",
    model: str | None = None,
    max_chars: int = 6000,
    retries: int = 2,
    backoff: float = 1.0,
) -> Dict[str, Any]:
    """Wrapper klasifikasi teks -> dict hasil klasifikasi.

    Parameter:
    - provider: kunci provider (mock|openai|deepseek|... di masa depan)
    - model: override nama model; jika None pakai default_model dari registry
    """
    cleaned = (text or "").strip()
    if not cleaned:
        return {"topik":"Lainnya","subtopik":"","sentimen":"netral","alasan_sentimen":"Teks kosong.","confidence":0.0}

    cleaned = cleaned[:max_chars]
    user_msg = USER_TEMPLATE.format(text=cleaned)
    messages = [
        {"role":"system","content":SYSTEM_PROMPT},
        {"role":"user","content":user_msg},
    ]

    prov_key, prov_cfg = get_provider_config(provider)
    model_name = model or prov_cfg["default_model"]

    if prov_key == "mock":
        # langsung pakai fungsi mock -> hasil dict
        return call_mock_classifier(cleaned)

    caller = prov_cfg["caller"]
    last_err = None
    for attempt in range(retries + 1):
        try:
            raw = caller(messages, model_name)
            # Jika provider bukan mock, raw biasanya string; pastikan parse
            if isinstance(raw, dict):  # kalau ada provider lain mengembalikan dict langsung
                parsed = raw
            else:
                parsed = parse_first_json_block(str(raw))
            return {
                "topik": parsed.get("topik","Lainnya"),
                "subtopik": parsed.get("subtopik",""),
                "sentimen": parsed.get("sentimen","netral"),
                "alasan_sentimen": parsed.get("alasan_sentimen",""),
                "poin_of_interest": parsed.get("poin_of_interest",""),
                "statement_pejabat": parsed.get("statement_pejabat",""),
                "confidence": float(parsed.get("confidence",0.0)),
            }
        except Exception as e:
            last_err = e
            logging.warning(f"Attempt {attempt+1} provider={prov_key} gagal: {e}")
            time.sleep(backoff * (2 ** attempt))

    # fallback ke mock kalau semua gagal
    fb = call_mock_classifier(cleaned)
    fb["alasan_sentimen"] += f" (fallback error: {last_err})"
    return fb

In [48]:
def run_pipeline_on_df(df: pd.DataFrame,
                       text_col: str = "artikel_berita",
                       title_col: str = "judul_berita",
                       provider: str = "mock",
                       sleep_sec: float = 0.0,
                       show_progress: bool = True,
                       classifier=None,
                       parallel: bool = True,
                       max_workers: int = 4,
                       cache_enabled: bool = True) -> pd.DataFrame:
    """
    Pipeline lengkap: bersihkan teks -> klasifikasi (LLM/mock) -> tambah kolom hasil.

    Optimasi tambahan:
    - parallel: jika True dan provider deepseek/openai maka gunakan ThreadPoolExecutor
      (I/O bound HTTP) dengan jumlah worker terbatas.
    - cache_enabled: aktifkan memoization berdasarkan hash teks untuk hindari panggilan ulang
      pada artikel identik (misal duplikasi). Hash memakai first 300 chars + length.
    - sleep_sec: jeda antar batch (bukan antar item) jika parallel.
    """
    from concurrent.futures import ThreadPoolExecutor, as_completed
    import hashlib

    # Siapkan classifier default
    if classifier is None:
        def _default_classifier(text: str):
            try:
                return classify_with_llm(text, provider=provider)
            except NameError:
                try:
                    return call_mock_classifier(text)
                except NameError:
                    t = (text or "").lower()
                    if any(k in t for k in ["apbn", "pajak", "kemenkeu", "sri mulyani", "bea cukai"]):
                        topik = "Kemenkeu"
                    elif any(k in t for k in ["presiden", "dpr", "kpk", "polri", "politik"]):
                        topik = "Nasional"
                    else:
                        topik = "Lainnya"
                    return {
                        "topik": topik,
                        "subtopik": "",
                        "sentimen": "netral",
                        "alasan_sentimen": "Fallback inline classifier.",
                        "confidence": 0.0,
                    }
        classifier = _default_classifier

    # Cache sederhana in-memory
    cache: dict[str, dict] = {}

    def _hash_text(s: str) -> str:
        # cukup sebagian awal untuk speed, plus panjang untuk beda
        head = s[:300]
        return hashlib.sha1((head + str(len(s))).encode('utf-8')).hexdigest()

    # Precompute cleaned text list
    cleaned_rows: list[tuple[int, str]] = []
    for idx, row in df.iterrows():
        raw_text = str(row.get(text_col, "") or "")
        if len(raw_text) < 100:
            raw_text = str(row.get(title_col, "") or "")
        cleaned = clean_article(raw_text)
        cleaned_rows.append((idx, cleaned))

    results_map: dict[int, dict] = {}

    # Fungsi eksekusi satu item dengan cache
    def _process(idx_cleaned: tuple[int, str]):
        idx_i, cleaned_i = idx_cleaned
        if cache_enabled:
            h = _hash_text(cleaned_i)
            if h in cache:
                return idx_i, cache[h]
        try:
            res = classifier(cleaned_i)
        except Exception as e:
            res = {
                "topik": "Lainnya",
                "subtopik": "",
                "sentimen": "netral",
                "alasan_sentimen": f"error: {str(e)[:200]}",
                "confidence": 0.0,
            }
        if cache_enabled:
            cache[h] = res
        return idx_i, res

    use_parallel = parallel and provider.lower() in {"deepseek", "openai"} and len(cleaned_rows) > 1

    if use_parallel:
        iterator = cleaned_rows
        if show_progress:
            iterator = tqdm(cleaned_rows, total=len(cleaned_rows), desc=f"Analisis berita (parallel {provider})", unit="berita", leave=True)
        with ThreadPoolExecutor(max_workers=max_workers) as ex:
            future_map = {ex.submit(_process, item): item[0] for item in iterator}
            for fut in as_completed(future_map):
                idx_i, res = fut.result()
                results_map[idx_i] = res
        if sleep_sec > 0:
            time.sleep(sleep_sec)
    else:
        iterator = cleaned_rows
        if show_progress:
            iterator = tqdm(cleaned_rows, total=len(cleaned_rows), desc=f"Analisis berita ({provider})", unit="berita", leave=True)
        for item in iterator:
            idx_i, res = _process(item)
            results_map[idx_i] = res
            if sleep_sec > 0:
                time.sleep(sleep_sec)

    # Susun output DF ternormalisasi
    output_records = []
    for idx in df.index:
        res = results_map.get(idx, {})
        original_cleaned = next((c for i,c in cleaned_rows if i==idx), "")
        output_records.append({
            "topik_llm": res.get("topik", "Lainnya"),
            "subtopik_llm": res.get("subtopik", ""),
            "sentimen": res.get("sentimen", "netral"),
            "alasan_sentimen": res.get("alasan_sentimen", ""),
            "poin_of_interest": res.get("poin_of_interest",""),
            "statement_pejabat": res.get("statement_pejabat",""),
            "confidence": res.get("confidence", 0.0),
            "text_len": len(original_cleaned),
        })

    out_df = pd.DataFrame(output_records, index=df.index)
    return pd.concat([df, out_df], axis=1)

# *OUTPUT*

## Eksekusi

<!-- Catatan Optimasi -->
Jika menggunakan provider DeepSeek, pipeline sekarang otomatis:
- Menggunakan parallel threads (I/O bound) default 4 worker.
- Reuse HTTP session (keep-alive) untuk mengurangi overhead koneksi.
- Mengurangi max_tokens output (JSON kecil) agar respons lebih cepat.
- Cache in-memory untuk teks identik (hindari panggilan ulang).

Parameter baru di run_pipeline_on_df:
- parallel (bool)
- max_workers (int)
- cache_enabled (bool)

Bisa override misalnya:
result_df = run_pipeline_on_df(df, provider=provider_selected, parallel=True, max_workers=6)

In [49]:
# Ambil parameter AI_name dari config
provider_selected = cfg["AI_name"]
provider_selected

'openai'

In [50]:
# 1. Load data berita & jalankan pipeline

# Ambil preferensi model jika ada di config
custom_model = cfg.get("AI_model_override") if isinstance(cfg, dict) else None
print("Provider yang dipakai:", provider_selected)

# Set parameter paralel khusus deepseek (I/O lebih lambat) -> percepat total waktu
parallel_args = {}
if provider_selected.lower() == 'deepseek':
    parallel_args = {"parallel": True, "max_workers": 6, "cache_enabled": True}
else:
    # untuk model cepat, masih bisa pakai parallel tapi default cukup
    parallel_args = {"parallel": True, "max_workers": 4, "cache_enabled": True}

# Jalankan pipeline untuk bersihkan teks & analisis topik/sentimen
result_df = run_pipeline_on_df(
    df,
    text_col="artikel_berita_bersih",
    title_col="judul_berita",
    provider=provider_selected,   # ganti dengan "openai" atau "deepseek" kalau pakai API
    show_progress=True, # tampilkan progress bar
    **parallel_args
)

if custom_model:
    print("Model override:", custom_model)

# Simpan hasil analisis full
today = pd.Timestamp.today().strftime("%Y%m%d")
model_suffix = custom_model or 'default'
output_path = f"analisis_ai_{today}_{provider_selected}_{model_suffix}.csv"
result_df.to_csv(output_path, index=False)
print(f"Pipeline selesai. Hasil disimpan di {output_path}")

Provider yang dipakai: openai


Analisis berita (parallel openai):   0%|          | 0/10 [00:00<?, ?berita/s]

2025-09-28 16:34:09,545 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-28 16:34:10,711 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-28 16:34:11,193 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-28 16:34:11,577 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-28 16:34:13,550 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-28 16:34:14,714 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-28 16:34:14,986 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-28 16:34:18,157 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-28 16:34:18,159 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "

Pipeline selesai. Hasil disimpan di analisis_ai_20250928_openai_default.csv


In [33]:
# %%
import json
from pathlib import Path
import logging

# Gunakan path lokal default (bukan path Colab) agar tidak error di Mac
CONFIG_PATH = Path("config.json")

logger = logging.getLogger(__name__)

def update_config(path: Path, new_values: dict):
    """Update config.json hanya pada key tertentu tanpa menimpa keseluruhan isi."""
    data = {}
    if path.exists():
        try:
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)
        except Exception as e:
            logger.warning(f"Gagal membaca config lama: {e}")
            data = {}

    # update hanya key yang diberikan
    data.update(new_values)

    try:
        with open(path, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        logger.info(f"Berhasil update config.json di {path}")
    except Exception as e:
        logger.error(f"Gagal menyimpan config.json: {e}")

# Simpan OUTPUT ke config dengan nama yang lebih jelas
update_payload = {
    "analisis_ai_output": output_path,
    "AI_name": provider_selected,
}
if cfg.get("AI_model_override"):
    update_payload["AI_model_override"] = cfg.get("AI_model_override")
update_config(CONFIG_PATH, update_payload)

2025-09-28 16:31:20,415 | INFO | Berhasil update config.json di config.json


In [34]:
# Sanity check: panggil LLM langsung untuk verifikasi (jika bukan mock)
sample_text = "Menteri Keuangan menyampaikan kebijakan fiskal terbaru terkait pajak dan APBN."
try:
    if provider_selected != 'mock':
        test_res = classify_with_llm(sample_text, provider=provider_selected, model=cfg.get("AI_model_override"))
        print(f"Hasil uji LLM ({provider_selected}):", test_res)
    else:
        print("Provider mock: skip panggilan API langsung.")
except Exception as e:
    print("Gagal memanggil LLM:", e)

Hasil uji LLM (deepseek): {'topik': 'Kemenkeu', 'subtopik': 'kebijakan fiskal pajak APBN', 'sentimen': 'netral', 'alasan_sentimen': 'Berita menyampaikan informasi kebijakan tanpa konteks penindakan atau kata kunci yang mengarah ke sentimen tertentu.', 'poin_of_interest': 'Menteri Keuangan', 'statement_pejabat': '', 'confidence': 0.9}
